In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [2]:
loader = TextLoader('video_transcription/final_file')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
docs = text_splitter.split_documents(docs)
db = Chroma.from_documents(docs)

Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [47]:
query = "Explain the maths behind a neural network"

In [48]:
ans = db.similarity_search(query,k=5)
context = ""
for i in range(5):
    context += ans[i].page_content
    context += "\n"

In [49]:
context

"an input and they take the weights of a\nneural network as an input and it's a\nmathematical expression and the output\nare your predictions of your neural net\nor the loss function we'll see this in a\nbit but basically neural networks just\nhappen to be a certain class of\nmathematical expressions\nbut back propagation is actually\nsignificantly more general it doesn't\nactually care about neural networks at\nall it only tells us about arbitrary\nmathematical expressions and then we\nhappen to use that machinery for\ntraining of neural networks now one more\nnote i would like to make at this stage\nis that as you see here micrograd is a\nscalar valued auto grant engine so it's\nworking on the you know level of\nindividual scalars like negative four\nand two and we're taking neural nets and\nwe're breaking them down all the way to\nthese atoms of individual scalars and\nall the little pluses and times and it's\njust excessive and so obviously you\nwould never be doing any of this in\

In [50]:
llm = OpenAI(temperature=0)
template="""Please use the following context to answer questions.
Context: {ans}
Question: {query}
Answer:"""

In [52]:
prompt = PromptTemplate(template=template, input_variables=["ans","query"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain.predict(ans = context, query = query)

' Neural networks are mathematical expressions that take input as data and weights and parameters of the neural net. The output of the neural network is a prediction or a loss function. Backpropagation is used to calculate the gradient of the loss function with respect to the parameters of the neural network. This allows us to tune the parameters of the neural network to decrease the loss locally.'